In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17644632
paper_name = 'cheng_bakalinsky_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/Supplementary_Table_1.xlsx', sheet_name='Table 1', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 250 x 17


In [10]:
original_data.head()

,Gene1,Yeast ORF,Minimum inhibitory concentration (mM)2\nOxalate Malonate Formate Acetate Propionate,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,S/C3,Function,Unnamed: 10,Unnamed: 11,Arabidopsis\nAccession4 E-value5,Unnamed: 13,Unnamed: 14,Sclerotinia\nAccession4 E-value5,Unnamed: 16
0,Transport:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARL1,YBR164C,10.0,105.0,10.0,30.0,20.0,NaN,n/n,Soluble GTPase with role in membrane traffic,NaN,NaN,At2g24765,NaN,4.000000e-54,SS1G01065.1,1.00E-36
2,ARP5,YNL059C,10.0,105.0,10.0,30.0,20.0,NaN,n/n,Actin-related protein involved in chromatin re...,NaN,NaN,At3g12380,NaN,2.000000e-70,SS1G13066.1,E-129
3,BAP3,YDR046C,10.0,105.0,10.0,40.0,20.0,NaN,n/n,Amino acid transporter,NaN,NaN,NaN,NaN,NaN,SS1G10461.1,3.00E-99
4,CCS1,YMR038C,10.0,105.0,10.0,30.0,20.0,NaN,n/n,SOD copper chaperone,NaN,NaN,At1g12520,NaN,8.000000e-20,SS1G00102.1,6.00E-32


In [11]:
original_data['orf'] = original_data['Yeast ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                         Gene1 Yeast ORF  \
index_input                                                                
0                                                   Transport:       NaN   
37                      Organelle organization and biogenesis:       NaN   
62                                 Vesicle-mediated transport:       NaN   
79                                             RNA metabolism:       NaN   
93                                                 Cell cycle:       NaN   
105                                            DNA metabolism:       NaN   
116                                             Transcription:       NaN   
126                                      Protein modification:       NaN   
136                                      Protein biosynthesis:       NaN   
137          ANP1        YEL036C               15          ...       NaN   
145                                          Cell homeostasis:       NaN   
153         

In [15]:
original_data = original_data.loc[t,:]

In [19]:
original_data['data'] = original_data['Minimum inhibitory concentration (mM)2\nOxalate     Malonate     Formate    Acetate     Propionate'].astype(float)

In [20]:
original_data.set_index('orf', inplace=True)

In [21]:
original_data = original_data[['data']].copy()

In [22]:
original_data = original_data.groupby(original_data.index).mean()

In [23]:
original_data.shape

(105, 1)

# Load & process tested strains

In [24]:
tested = pd.read_excel('raw_data/YSC1054Y.copy.xlsx', sheet_name='mat_alpha_obs')

In [25]:
tested.head()

,record no.,ORF name,strain,batch,plate,row,column,notes
0,10405.0,YAL002W,BY4739,01_1,101.0,E,1,NaN
1,10402.0,YAL004W,BY4739,01_1,101.0,D,11,NaN
2,10403.0,YAL005C,BY4739,01_1,101.0,D,12,NaN
3,10401.0,YAL007C,BY4739,01_1,101.0,D,10,NaN
4,10400.0,YAL008W,BY4739,01_1,101.0,D,9,NaN


In [26]:
tested['orf'] = tested['ORF name'].astype(str)

In [27]:
tested['orf'] = clean_orf(tested['orf'])

In [28]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [29]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name strain batch  plate  row   column notes  orf
index_input                                                                  
4826                NaN      NaN    NaN   NaN    NaN  NaN  end 150   NaN  NAN


In [30]:
tested = tested.loc[t,:]

In [31]:
tested_orfs = tested['orf'].unique()

In [32]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [33]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [34]:
data = original_data.copy()

In [35]:
dataset_ids = [114]
datasets = datasets.reindex(index=dataset_ids)

In [36]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [37]:
data.head()

dataset_id,114
data_type,value
orf,
YAL002W,0.0
YAL004W,0.0
YAL005C,0.0
YAL007C,0.0
YAL008W,0.0


## Subset to the genes currently in SGD

In [38]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [39]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,114
,data_type,value
gene_id,orf,
2,YAL002W,0.0
1863,YAL004W,0.0
4,YAL005C,0.0
5,YAL007C,0.0
6,YAL008W,0.0


# Normalize

In [40]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [41]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [42]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        114       
data_type       value valuez
gene_id orf                 
2       YAL002W   0.0    0.0
1863    YAL004W   0.0    0.0
4       YAL005C   0.0    0.0
5       YAL007C   0.0    0.0
6       YAL008W   0.0    0.0

# Print out

In [43]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [44]:
from IO.save_data_to_db3 import *

In [45]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 17644632...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.01s/it]

Updating the data_modified_on field...
